In [1]:
# Import packages
%run startup.py
# Load all questions
load_questions()

In [4]:
# initialize network and snapshot
NETWORK_NAME = "forwarding-change-validation"
BASE_NAME = "base"
BASE_PATH = "networks/forwarding-change-validation/base"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(BASE_PATH, name=BASE_NAME, overwrite=True)
# bf_set_snapshot(name = BASE_NAME)

'base'

In [3]:
df = bfq.routes().answer(snapshot=BASE_NAME).frame()

In [5]:
CHANGE_NAME = "change"
CHANGE_PATH = "networks/forwarding-change-validation/change1"
# bf_set_snapshot(name=CHANGE_NAME)
bf_init_snapshot(CHANGE_PATH,name = CHANGE_NAME, overwrite=True)

'change'

In [5]:
# invoke bfq.routes to push batfish parse config files and compute data plane
change_routes = bfq.routes().answer(snapshot = CHANGE_NAME).frame()

In [26]:
result = bfq.detectLoops().answer(snapshot=CHANGE_NAME).frame()
show(result)

,Flow,Traces,TraceCount
0,Start Location: border2Src IP: 2.1.1.2Dst IP: 10.12.11.2IP Protocol: ICMP,"LOOP1. node: border2 ORIGINATED(default) FORWARDED(Routes: ospfE2 [Network: 10.12.11.0/24, Next Hop IP:2.12.22.2]) TRANSMITTED(GigabitEthernet1/0)2. node: core2 RECEIVED(GigabitEthernet0/0) FORWARDED(Routes: static [Network: 10.12.11.2/32, Next Hop IP:2.23.22.3]) TRANSMITTED(GigabitEthernet2/0)3. node: spine2 RECEIVED(GigabitEthernet0/0) FORWARDED(Routes: static [Network: 10.12.11.2/32, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)4. node: leaf1 RECEIVED(GigabitEthernet1/0) PERMITTED(RESTRICT_NETWORK_TRAFFIC_IN (INGRESS_FILTER)) FORWARDED(Routes: static [Network: 10.12.11.2/32, Next Hop IP:2.34.101.3]) TRANSMITTED(GigabitEthernet0/0)5. node: spine1 RECEIVED(GigabitEthernet2/0) FORWARDED(Routes: static [Network: 10.12.11.2/32, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet0/0)6. node: core1 RECEIVED(GigabitEthernet2/0) FORWARDED(Routes: ospfE2 [Network: 10.12.11.0/24, Next Hop IP:2.12.21.1]) TRANSMITTED(GigabitEthernet1/0)7. node: border2 RECEIVED(GigabitEthernet2/0)",1


In [1]:
from client.apkeep import RealConfig
import pandas as pd
%run startup.py
from IPython.display import display,HTML

In [2]:
# Initialize an apkeep model based on the original configuration
realConfig = RealConfig('forwarding-change-validation','base')

In [3]:
ans1 = realConfig.base_check()

loops : 0, differentialReachability : 12


In [4]:
ans1.differentialReachability()

,flows,before paths,after paths
0,src ip:2.128.0.0-2.128.255.255dst ip:2.128.0.0-2.128.0.1src port:0-65535dst port:0-65535protocol:17-17,"1(1).border2RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(None)","1(1).border2RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(2).core1RECEIVED(GigabitEthernet1/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(3).spine1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(4).leaf1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(5).host-dbRECEIVED(eth0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(None)"
1,src ip:2.128.0.0-2.128.255.255dst ip:2.128.0.0-2.128.0.1src port:0-65535dst port:0-65535protocol:17-17,"1(1).border1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(None)","1(1).border1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet1/0)(2).core1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(3).spine1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(4).leaf1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(5).host-dbRECEIVED(eth0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(None)"
2,src ip:2.129.0.0-255.255.255.255dst ip:2.128.1.1-2.128.1.1src port:0-65535dst port:22-22protocol:6-6,"1(1).border2RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(None)","1(1).border2RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(2).core1RECEIVED(GigabitEthernet1/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(3).spine1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(4).leaf1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet3/0)(5).host-wwwRECEIVED(eth0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(None)"
3,src ip:2.128.0.0-2.128.255.255dst ip:2.128.1.1-2.128.1.1src port:0-65535dst port:0-65535protocol:17-17,"1(1).border1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(None)","1(1).border1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet1/0)(2).core1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(3).spine1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(4).leaf1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet3/0)(5).host-wwwRECEIVED(eth0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(None)"
4,src ip:0.0.0.0-2.127.255.255dst ip:2.128.0.0-2.128.0.1src port:0-65535dst port:0-65535protocol:17-17,"1(1).border1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANS

In [5]:
ans2 = realConfig.update_check('change')

loops : 0, differentialReachability : 4


In [6]:
ans2.differentialReachability()

,flows,before paths,after paths
0,src ip:2.128.0.0-2.128.255.255dst ip:2.128.1.1-2.128.1.1src port:0-65535dst port:0-65535protocol:17-17,"1(1).border1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet1/0)(2).core1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(3).spine1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(4).leaf1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet3/0)(5).host-wwwRECEIVED(eth0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(None)","1(1).border1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(2).core2RECEIVED(GigabitEthernet1/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(null)"
1,src ip:2.129.0.0-255.255.255.255dst ip:2.128.1.1-2.128.1.1src port:0-65535dst port:22-22protocol:6-6,"1(1).border1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet1/0)(2).core1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(3).spine1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(4).leaf1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet3/0)(5).host-wwwRECEIVED(eth0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(None)","1(1).border1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(2).core2RECEIVED(GigabitEthernet1/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(null)"
2,src ip:2.129.0.0-255.255.255.255dst ip:2.128.1.1-2.128.1.1src port:0-65535dst port:22-22protocol:6-6,"1(1).border2RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(2).core1RECEIVED(GigabitEthernet1/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(3).spine1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(4).leaf1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet3/0)(5).host-wwwRECEIVED(eth0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(None)","1(1).border2RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet1/0)(2).core2RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(null)"
3,src ip:2.128.0.0-2.128.255.255dst ip:2.128.1.1-2.128.1.1src port:0-65535dst port:0-65535protocol:17-17,"1(1).border2RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(2).core1RECEIVED(GigabitEthernet1/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(3).spine1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet2/0)(4).leaf1RECEIVED(GigabitEthernet0/0)NONE(None)FORWARDED(Routes:(Prefix:None,Next Hop Interface:None))NONE(None)TRANSMITTED(GigabitEthernet3/0)(5).host-wwwRECEIVED(eth0)NONE(None)FORWARDED(Rout